## DVCS Tool Prototypes

These will eventually move into some dedicated files.

In [ ]:
import ipywidgets as W, traitlets as T, pathlib as P, time, pprint
from datetime import datetime

In [ ]:
def make_repo_explorer():
    return W.HBox([
        W.Label("History"),
        W.Select(options=["origin", "collaborator1"], description="remote", multiple=False, rows=1),
        W.Select(options=["main", "dev"], description="head", multiple=False, rows=1)
    ])
make_repo_explorer()

In [ ]:
def make_actions():
    return W.VBox([
        W.Label("Actions"), 
        W.HBox([
            W.Label("Save"), 
            W.HTML("<code>+100/-75</code>", description="Δ"),
            W.Text(description="Message", placeholder="Updated value.yaml"),
            W.Button(description="Save", icon="check")
        ]), 
        W.HBox([
            W.Label("Restore"), 
            W.Select(options=["main", "dev"], description="head", multiple=False, rows=1),
            W.SelectionSlider(options=["just now", "5 minutes ago"], description="history", multiple=False, rows=1),
            W.HTML("<code>+100/-75</code>", description="Δ"),
            W.Button(description="Revert", icon="refresh")
        ]),
    ])
make_actions()

In [ ]:
def make_basic_repo_ui(repo):
    watch = W.Checkbox(True, description="Watch")
    changes = W.HTML()
    message = W.Text(placeholder="Commit message")
    commit = W.Button(description="Commit")
    
    def _do_commit(*args):
        staged = []
        for change in repo.changes:
            for path in ["a_path", "b_path"]:
                if change[path] and change[path] not in staged:
                    repo.stage(change[path])
                    staged += [change[path]]
        repo.commit(message.value or message.placeholder)
    
    commit.on_click(_do_commit)
    
    def change_html(changes):
        if not changes:
            message.placeholder = "Commit message"
            return "<i>Δ0</i>"
        paths = sum([[change["a_path"], change["b_path"]] for change in changes], [])
        paths = sorted(set([p for p in paths if p]))
        message.placeholder = f"""Updated {", ".join(paths)}"""

        return f"""<i title="{message.placeholder}">Δ{len(changes)}</i>"""
    
        
    T.link((watch, "value"), (repo, "watching"))
    T.dlink((repo, "changes"), (changes, "value"), change_html)
    T.dlink((repo, "changes"), (commit, "disabled"), lambda x: not len(x))
    return W.HBox([W.VBox([watch, changes]), W.VBox([message, commit])])

In [ ]:
def make_head_switcher(repo, description="Heads"):
    switcher = W.Dropdown(description=description)
    refresh = W.Button(icon="refresh")
    refresh.on_click(lambda *x: repo._update_heads())
    T.dlink((repo, "heads"), (switcher, "options"))
    return W.HBox([switcher, refresh])

In [ ]:
def default_history_label(c):
    return """{} by {} at {}""".format(
        c["message"],
        c["author"]["name"],
        datetime.fromtimestamp(c["timestamp"]).isoformat()
    )
    
def make_commit_switcher(repo, selection_cls=W.Dropdown, history_label=default_history_label):
    switcher = selection_cls()
    revert = W.Button(description="Revert")
    def _make_options(head_history):
        return [(history_label(c), c["commit"]) for c in head_history]
    T.dlink((repo, "head_history"), (switcher, "options"), _make_options)
    def _switch(self):
        repo.revert(switcher.value)
    revert.on_click(_switch)
    return W.HBox([switcher, revert])